# Graph Convolutional Neural Networks


In this notebook will learn about Graph Convolutional Neural Networks by implementing a simple GCN to classify the nodes of a graph.

**Important:** Set the Colab environment to run on GPU

**Notebook created by Paula Gómez Duran**


## **Installation . . .**

In [ ]:
%tensorflow_version 1.x
import torch
if not torch.cuda.is_available():
    raise Exception("You should enable GPU runtime")

In [ ]:
!pip install -q torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install -q torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.7.0.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

## **Understanding GCN concept by building a simple graph . . .**

<div>
<center><img src="https://miro.medium.com/max/864/1*jTW7doI_cqC_p9XQrmuu9A.png" width="400"/></center>
</div>

### Defining the graph and it's features

So, we build the adjacency matrix A from the previous figure:

In [ ]:
import numpy as np
#############################
# Exercice 1: Complete the last two rows of the matrix. Will or will not be symmetric matrix?
#############################
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    # [ ... ],
    # [ ... ]],
    dtype=float
)

Now, we generate 2 features for each node based on the index. This makes it easy to confirm the matrix calculations manually later.

In [ ]:
X = np.matrix([
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)
X

### Applying the simple propagation rule for the input layer:

Now we will try to implement the layer equation defined in theory. To make it easier, we will ommit the normalization part, as it is implicit in the implementation of the model from Pytorch Geometric that we will use and it is not necessary to get the intuition of how GCN works.


### ` f(Hⁱ, A) = σ(A Hⁱ Wⁱ) `

*   H⁰ = X = input features
*   AH⁰W⁰ = AXW⁰ = AX
*   W = I






In [ ]:
H_0 = X 

In [ ]:
W = np.identity(2)

In [ ]:
H_1 = A * H_0 * W
H_1

As you can observe, the representation of each node (each row) is now a sum of its neighbors features! 

In other words, the graph convolutional layer represents each node as an aggregate of its neighborhood, as we saw on theory slides.

In [ ]:
for i in range(1000):
    X = A * X * W
print(X)

### Adding self loops

If you look carefuly to the previous representation of each node after the popagation rule, the node itself is not taken into account. Thus, that is the reason why we need to add self loops to the adjacency matrix, conforming the Â matrix.


In [ ]:
#############################
# Exercice 2: Build identity matrix I using np.identity() 
#             Sum I with A matrix to build Â
#############################

I  = # ... 
I

In [ ]:
A_hat = A + I
A_hat

In [ ]:
# Now run again the GCN equation and observe the new output
H_1 = A_hat * H_0 * W
H_1

As you observe, now we have taken into account the node itself when computing the aggregation of local neightbours by A * X.

### Why do we need to normalize?




Here we intent to show what happens without normalizing. Then, we will show whats the same output when applying normalization.

> So, lets try to apply the GCN equation for 1000 epochs:

In [ ]:
H_n = H_1
for i in range(1000):
    H_n = A_hat * H_n * W

print(H_n)

WHY DO YOU THINK IT HAPPENS?

> You can observe that the output features for epoch 1000 are nan... So, let's try to normalize before and repeat the operation.

In [ ]:
A_hat

In [ ]:
D = np.array(A_hat.sum(1))
D

In [ ]:
D_inv_half = np.power(D, -0.5).flatten()
D_mat = np.diag(D_inv_half)
# SYMMETRIC NORMALIZATION = D^-1/2 * (H) * D^-1/2
aux = D_mat.dot(A_hat)
norm_A_hat = aux.dot(D_mat)
norm_A_hat

In [ ]:
H_1_norm = norm_A_hat * H_0 * W
H_1_norm

In [ ]:
H_n = H_1_norm
for i in range(1000):
    H_n = norm_A_hat * H_n * W

print(H_n)

> Now, as you observe we still can get good results after iterating for 1000 epochs. 


The explanation is that normalizing avoid numbers to be higher than one in all the multiplications, and so we need to do it in order to solve possible problems such as exploding gradients or having weights which are too big.

## **Moving to a real challenge . . .**

Now, we will explore how to solve a classification problem with GCN. 




The task that we will solve is based on **Cora dataset** ( [link here](https://relational.fit.cvut.cz/dataset/CORA) ), which consists of 2708 scientific publications classified into seven classes:

		Case_Based
		Genetic_Algorithms
		Neural_Networks
		Probabilistic_Methods
		Reinforcement_Learning
		Rule_Learning
		Theory

The papers were selected in a way such that every paper cites or is cited by atleast one other paper. The citation network consists of 5429 links which represents the edge of the graph. Therefore, we can assume that the data is describing in a graph structure.

Besides, the dataset also provides a 0/1-valued word vector for each of the papers which indicates the absence/presence of each corresponding word from the dictionary, which has a lenght of 1433 positions.

So, as features of each node we will have a vector of binary values indicating whether each word in the vocabulary is present (indicated by 1) or absent (indicated by 0) in the paper. I have put an example here for a better understanding of the concept:


<div>
<center><img src="https://miro.medium.com/max/906/1*f5e9vn4EZB8zNSLWO0dn-A.png"/></center>
</div>


So, assuming that the dictionary consists in those words (awesome, funny, hate, it ...), we would have for each of the papers (0, 1, 2, 3 in the example above), a 1 value in those positions where the corresponding word was present.


> **About dictionary:** After stemming and removing stopwords we were left with a vocabulary of size 1433 unique words. All words with document frequency less than 10 were removed.


Finally, they also provide labels for each of the papers which corresponds to a category that indicates the class of which the paper has been classified on.

 &nbsp;

**The goal is to classify each document into one of the seven classes**. In order to make it easier when working with graphs we will use the library **Pytorch Geometric**. You can find the documentation [here](https://pytorch-geometric.readthedocs.io/en/latest/) and it's github page [here](https://github.com/rusty1s/pytorch_geometric).

 &nbsp;
 

Now, we are going to import [**Tensorboard**](https://www.tensorflow.org/tensorboard/get_started), which is a tool for providing the measurements and visualizations needed during the machine learning workflow. It is very easy to use and it will allow us to see straightforward the behaviour of our network.

In [ ]:
%tensorflow_version 1.x
from tensorboardcolab import TensorBoardColab
tbc = TensorBoardColab()

Prepare the necessary imports:

In [ ]:
# Write code for imports
import os.path as osp
import argparse
import numpy as np

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  
device = torch.device("cuda")



Pytorch Geometric already implements **Planetoid** class, which loads **Cora** dataset into a very useful way for using GCN. Let's take a look!

In [ ]:
dataset_name = 'Cora'
path = osp.join(osp.dirname(osp.realpath('coradataset')), 'data', dataset_name)
# NormalizeFeatures --> Row-normalizes node features to sum-up to one.
dataset = Planetoid(path, dataset_name, transform=T.NormalizeFeatures())
dataset

In [ ]:
# Write code for visualizing dataset object
data = dataset[0]
data

In [ ]:
dataset


#### Understanding data...

As we previously said, Cora dataset contains bag-of-words representation of documents and citation links between the documents. Just above we can observe the attributs of the [*InMemoryDataset (Planetoid)*](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/planetoid.html#Planetoid) class from Pytorch Geometric library that will help us. Let's analyse each of them!

&nbsp;

We will treat the bag-of-words vectors as feature vectors **X**. As we can observe, the shape of *X* is the number of nodes on the rows and the lenght of the dicctonaty on the columns. So, we have a matrix which contains a feature vector for each of the nodes (one representation in each row).


In [ ]:
#############################
# Exercice 3: Write code for visualizing the shape of the input node features[Num_nodes, Dicctionary lenght]
#             TIP: take into account that `dataset` variable is different from `data` object. You might want to check on `data`.            
#############################

# ...


In [ ]:
# Visualize first feature vector (feature of node 0)

# ...

To construct the graph, we need to build the adjacency matrix *A* based on the citation links: if document *i* cites *j*, then we set *aij* = *aji* = 1. However, Pytorch Geometric gives us those relationships in the *edge_index* variable, which denotes each of the positive interactions of A. 

To make it clear, **A** will have shape (2708, 2708) in order to have both in rows and in columns all the nodes and so be able to set to 1 those positions where document *i* cited document *j*. The next variable **edge_index** will refer to all the positions set to one: 

      node 0 - node 633
      node 0 - node 1862
      node 0 - node 2582
              .
              .
              .
     node 2707 - node 1473
     node 2707 - node 2706



In [ ]:
# Print edge_index variable 
data.edge_index

If we check with how many nodes (papers) is related a given paper, we can observe that still there is a lot more which with it is not related. Thus, the matrix **A** will be a sparse matrix, which is basically a matrix contaning a lot of zeros. 


In [ ]:
# Print number of connections per node
for node in [1, 2, 3]:
    print(f'Number of connections for node {node} : {np.where(data.edge_index[0].numpy() == node)[0]}')

So, if you see above, the node 1 is connected with 3 nodes; the node 2 with 5 nodes; and the node 3 with one node. Thus, we can confirm that the majory of connections of **A** will be set to 0 - being **A** a sparse matrix.


> **Remember:** The diagonal of **A** will always be full of zeros before adding the self-loops (identity matrix).

#### SPLIT
If we look at *Planetoid class* [documentation](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/planetoid.html#Planetoid) that corresponds to the dataset we are using, we will see that there is a **split** flag where we can decide the way the dataset should be splitted. The library provides us three different ways to do it, as it is explained in [docs](https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/datasets/planetoid.html#Planetoid):


* "public" : the split will be the public fixed split from the "Revisiting Semi-Supervised Learning with Graph Embeddings" [paper](https://arxiv.org/abs/1603.08861).

* "full": all nodes except those in the validation and test sets will be used for training, as in the "FastGCN: Fast Learning with Graph Convolutional Networks via Importance Sampling" [paper](https://arxiv.org/abs/1801.10247).

* "random": train, validation, and test sets will be randomly generated, according to `num_train_per_class`, `num_val` and `num_test`. 


The default value of the split is the *public* one and so what the dataset returns are three different masks: one for training, one for validation and one for test. Each of the mask will have a true those indices selected for either training, validation or test.



In [ ]:
# Print the mask for training set
data.train_mask

In [ ]:
# Check training labels
data.y[data.train_mask]


We will let it set as the default option, which refers to the public split on their paper **"Revisiting Semi-Supervised Learning with Graph Embeddings"**. It consists in randomly sample 20 instances for each class as labeled data (20*7 = 140) and then evaluate it on 1000 instances as test data. The rest of data will be used for validation as unlabeled data. 

In [ ]:
# Check how many labels we have

len(data.y)

In [ ]:
# Print lenght of training, val and test samples
print(len(data.y[data.train_mask]), len(data.y[data.val_mask]), len(data.y[data.test_mask]))

In case you did not notice, we imported *GCNConv class* above and you can look at it's documentation [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html?highlight=GCNConv#torch_geometric.nn.conv.GCNConv). **GCNConv** implements the GCN operation we explained in the slides, which is basically a graph convolutional layer. To build the network, we will use two GCN layers thus combining them with some activation functions and dropout regularization. In the end, we will use a *log_softmax* function to get the class prediction probabilities. 


You can read the documentation for better understanding how this class works and what it is already implementing.

In [ ]:
# Build GCN model
#############################
# Exercice 4: Build two GCNConv layers that go from:
#             1. the number of features to hidden dimension
#             2. hidden dimension to number of classes
# Set cached = True for both layers
#############################

class GCN_network(torch.nn.Module):
    def __init__(self, num_features, num_classes, hidden_dim=16):
        super(GCN_network, self).__init__()
        self.conv1 = # ...
        self.conv2 = # ...
        
    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        # DEFAULT DO = 0.5
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        return F.log_softmax(x, dim=1)

Now we will declare the network as well as the optimizer with the needed parameters.

In [ ]:
# Declare model and optimizer. Also, move data to device.

model = GCN_network(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In Graphs, we do not have three different datasets (three different graphs). However, we have one entire graph and then the labels are boolean masks which allow us to select the nodes. Thus, the way to compute the predictions will be to predict the whole graph and select the indexes that we want either for train, val or test depending on the situation.

In [ ]:
# Predict for all data and check shape

entire_graph_pred = model(data)
entire_graph_pred.shape

In [ ]:
# Select just training set predictions

entire_graph_pred[data.train_mask].shape

Now we proceed to build the **train** and **validation** functions for training the network and evaluating it. Besides, we will also build the **test** function for performing inference with **accuracy** metric in each of the sets.

In [ ]:
# Write code for tranining function

def train():
    model.train()
    optimizer.zero_grad()
    train_pred = model(data)[data.train_mask]
    train_labels = data.y[data.train_mask]

    train_loss = F.nll_loss(train_pred, train_labels)
    train_loss.backward()
    tbc.save_value('loss/train', 'train_loss', epoch, train_loss.item())
    optimizer.step()

In [ ]:
# Write code for validation function

@torch.no_grad()
def validation():
    model.eval()
    #############################
    # Exercice 5: Take the validation indices of the entire graph predictions
    #############################
    
    val_pred = # ...
    val_labels = data.y[data.val_mask]

    val_loss = F.nll_loss(val_pred, val_labels)
    tbc.save_value('loss/val', 'val_loss', epoch, val_loss.item())


In [ ]:
# Write code for evaluation function
@torch.no_grad()
def test():
    model.eval()
    logits = model(data)
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):

        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

As the last stage, we will write the code for the entire workflow for 300 epochs.

1. Training
2. Validating
3. Inference

In [ ]:
best_val_acc = test_acc = 0
for epoch in range(1, 301):
    train()
    validation()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'

    tbc.save_value('acc/train','train_acc', epoch, train_acc)
    tbc.save_value('acc/val','val_acc', epoch, best_val_acc)
    tbc.save_value('acc/test','test_acc', epoch, test_acc)

    print(log.format(epoch, train_acc, best_val_acc, test_acc))

Now we have trained the network and evaluated it! You can go to the Tensorboard link generated above and you will be able to see that actually, we did not need as much epochs because the model is overfitting in approximately epoch 90 (validation loss stops going down). Even though, we have achieved a pretty good performance on test accuracy (0.7910). 
